In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np

import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

df_1 = pd.read_csv("../output/task2-1.csv")
df_2 = pd.read_csv("../output/task2-2.csv")
df_2_total = pd.read_csv("../output/task2-2-total.csv")

In [2]:
# plot the number of cells over time
scaled_time = df_1['time'] * 0.001
plt.figure(figsize=(8, 2))
plt.plot(scaled_time, df_1['cells'], label='Cells', color='blue')
plt.axvline(x=1200, color='black', linestyle='dashed')
plt.xlabel('Time')
plt.ylabel('Number of Cells')
plt.savefig('../../report/graphics/task2-1.pdf', bbox_inches='tight')
plt.close()

In [3]:
plt.figure(figsize=(3, 3))
plt.plot(df_2['x'], df_2['y'], label='_task2-2', color='gray')
plt.plot(df_2['x'][0], df_2['y'][0], 'bx', label='Start', ms=10, mew=2)
plt.plot(df_2['x'].iloc[-1], df_2['y'].iloc[-1], 'rx', label='End', ms=10, mew=2)
plt.xlim(25,75)
plt.ylim(25,75)
plt.grid()
plt.margins(0)
plt.legend(loc='upper left')
plt.savefig('../../report/graphics/task2-2.pdf', bbox_inches='tight')
plt.close()

In [4]:
# Load all .csv file
data = {}
for file in os.listdir("../output/bulk-m"):
    if file.endswith(".csv"):
        name = file.split(".")[0]
        record = {}
        record["df"] = pd.read_csv(f"../output/bulk-m/{file}")
        # log to base 10 for power
        record["m"] = int(name[8:])
        data[name] = record

# sort by m
data = dict(sorted(data.items(), key=lambda x: x[1]["m"]))

for name, record in data.items():
    dt = 0.001
    record['df']['time'] = record['df']['time'] * dt

# calculate the percentage of cells that are filled at t=1000 for each m and print to console like table
print("M & Cells & Percentage \\\\")
all_m = []
percentages_sim = []
percentage_analytical = []
for name, record in data.items():
    m = record['m']
    cells = record['df'].iloc[-1]['cells']
    print(f"{m} & {(cells/10 ** m):.4g} & {10 ** m} \\\\")
    percentages_sim.append(cells/10 ** m)
    all_m.append(m)

    # calculate analytical solution
    percentage_analytical.append(10 ** (m - ((m - 9) * (np.e ** (-0.006 * 1200)))) / 10 ** m)
print()

plt.figure(figsize=(8, 2))
plt.plot(all_m, percentages_sim, color='blue', label='Simulation')
plt.plot(all_m, percentage_analytical, color='red', linestyle='dashed', label='Analytical')
plt.xlabel('M')
plt.ylabel('Percentage of Cells')
plt.legend()
plt.savefig('../../report/graphics/task2-1-m-bar-percentage.pdf', bbox_inches='tight')

# for each m, solve anylitical solution and calculate mean squared error percentage
print("M & RMSE \\\\")
plt.figure(figsize=(8, 2))
for name, record in data.items():
    m = record['m']
    y = 10 ** (m - ((m - 9) * (np.e ** (-0.006 * record['df']['time']))))

    # calculate root mean square percentage error
    rsme = np.sqrt(np.mean((record['df']['cells'] - y) ** 2)) / np.mean(y) * 100

    print(f"{m} & {rsme:.4g} \\\\")

    # bar chart
    plt.bar(m, rsme, color='blue')
print()
plt.xlabel('M')
plt.ylabel('RMSE')
plt.savefig('../../report/graphics/task2-1-m-bar-error.pdf', bbox_inches='tight')

M & Cells & Percentage \\
9 & 1 & 1000000000 \\
10 & 0.9983 & 10000000000 \\
11 & 0.9966 & 100000000000 \\
12 & 0.9949 & 1000000000000 \\
13 & 0.9931 & 10000000000000 \\
14 & 0.9914 & 100000000000000 \\
15 & 0.9897 & 1000000000000000 \\
16 & 0.988 & 10000000000000000 \\
17 & 0.9863 & 100000000000000000 \\

M & RMSE \\
9 & 0 \\
10 & 2.857e-05 \\
11 & 0.0001459 \\
12 & 0.0003193 \\
13 & 0.0005233 \\
14 & 0.0007504 \\
15 & 0.000997 \\
16 & 0.001261 \\
17 & 0.00154 \\



In [5]:
# plot the number of cells over time for all m
plt.figure(figsize=(8, 3))
for name, record in data.items():
    plt.plot(record['df']['time'], record['df']['cells'], label=('$10^{' + str(record["m"]) + '}$'))
plt.xlabel('Time')
plt.ylabel('Number of Cells')
plt.legend(title='Capacity')
plt.yscale('log')
plt.ylim(1e8, 1e17)
plt.savefig('../../report/graphics/task2-1-m.pdf', bbox_inches='tight')

/var/folders/y0/2k6mgl4x5_db4cbjlvjsc0jw0000gn/T/ipykernel_87248/2561895735.py:10: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig('../../report/graphics/task2-1-m.pdf', bbox_inches='tight')
/var/folders/y0/2k6mgl4x5_db4cbjlvjsc0jw0000gn/T/ipykernel_87248/2561895735.py:10: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig('../../report/graphics/task2-1-m.pdf', bbox_inches='tight')


In [6]:
# plot the number of cells over time for all m
plt.figure(figsize=(8, 3))
for name, record in data.items():
    plt.plot(record['df']['time'], record['df']['cells'] / 10 ** record["m"], label=('$10^{' + str(record["m"]) + '}$'))
plt.xlabel('Time')
plt.ylabel('Percentage of Cells')
plt.legend(title='Capacity')
plt.savefig('../../report/graphics/task2-1-m-percent.pdf', bbox_inches='tight')

In [7]:
# Load all .csv file
data = {}
for file in os.listdir("../output/bulk-h"):
    if file.endswith(".csv"):
        name = file.split(".")[0]
        record = {}
        record["df"] = pd.read_csv(f"../output/bulk-h/{file}")
        # log to base 10 for power
        record["h"] = int(name[10:])
        data[name] = record

# scale time
for name, record in data.items():
    dt = 10 ** record['h']
    record['df']['time'] = record['df']['time'] * dt

# sort by h
data = dict(sorted(data.items(), key=lambda x: x[1]['h'], reverse=True))

# solve analytically first
x = np.linspace(0, 1200)
y = 10 ** (13 - (4 * (np.e ** (-0.006 * x))))
analytical = pd.DataFrame({'time': x, 'cells': y})

# calculate the error to the analytical solution for all h using root mean square percentage error
for name, record in data.items():
    record['error'] = np.sqrt(np.mean((record['df']['cells'] - np.interp(record['df']['time'], analytical['time'], analytical['cells'])) ** 2)) / np.mean(record['df']['cells']) * 100

# print in table format
print("h & Error")
h_mse = []
for name, record in data.items():
    # round to 5 significant figures
    print(f"{10 ** record['h']} & {record['error']:.4}\% \\\\")
    h_mse.append(record['error'])
print("Analytical & 0.0 \\\\")
print()

plt.figure(figsize=(8, 2))
plt.plot([10 ** record['h'] for name, record in data.items()], h_mse, color='blue', label='Simulation')
plt.xlabel('H')
plt.ylabel('RMSE')
plt.savefig('../../report/graphics/task2-1-h-bar-error.pdf', bbox_inches='tight')
    

h & Error
100 & 66.65\% \\
10 & 5.294\% \\
1 & 0.5252\% \\
0.1 & 0.06732\% \\
0.01 & 0.04269\% \\
0.001 & 0.04237\% \\
0.0001 & 0.04236\% \\
Analytical & 0.0 \\



In [8]:

# plot the number of cells over time for all h
plt.figure(figsize=(8, 2.5))
for name, record in data.items():
    plt.plot(record['df']['time'], record['df']['cells'], label=f'{10 ** record["h"]}')
plt.plot(analytical['time'], analytical['cells'], label='Analytical', color='black', linestyle='dashed')
plt.xlabel('Time')
plt.ylabel('Number of Cells')
plt.legend(title='h')
plt.savefig('../../report/graphics/task2-1-h.pdf', bbox_inches='tight')

/var/folders/y0/2k6mgl4x5_db4cbjlvjsc0jw0000gn/T/ipykernel_87248/483870939.py:9: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig('../../report/graphics/task2-1-h.pdf', bbox_inches='tight')
/var/folders/y0/2k6mgl4x5_db4cbjlvjsc0jw0000gn/T/ipykernel_87248/483870939.py:9: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig('../../report/graphics/task2-1-h.pdf', bbox_inches='tight')


In [9]:
# plot again but zoomed in ar 450-550
plt.figure(figsize=(8, 2.5))
for name, record in data.items():
    plt.plot(record['df']['time'], record['df']['cells'], label=f'{10 ** record["h"]}')
plt.plot(analytical['time'], analytical['cells'], label='Analytical', color='black', linestyle='dashed')
plt.xlabel('Time')
plt.ylabel('Number of Cells')
plt.legend(title='h')
plt.xlim(450, 550)
plt.ylim(0.55e13, 0.65e13)
plt.savefig('../../report/graphics/task2-1-h-zoom.pdf', bbox_inches='tight')
plt.close()

/var/folders/y0/2k6mgl4x5_db4cbjlvjsc0jw0000gn/T/ipykernel_87248/2428944605.py:11: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig('../../report/graphics/task2-1-h-zoom.pdf', bbox_inches='tight')
/var/folders/y0/2k6mgl4x5_db4cbjlvjsc0jw0000gn/T/ipykernel_87248/2428944605.py:11: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig('../../report/graphics/task2-1-h-zoom.pdf', bbox_inches='tight')


In [10]:
# plot the number of cells over time
plt.figure(figsize=(8, 2))
plt.plot(df_2_total['time'], df_2_total['cells'], label='Cells', color='gray')
plt.xlabel('Time')
plt.ylabel('Number of Cells')
plt.savefig('../../report/graphics/task2-2-total.pdf', bbox_inches='tight')

In [11]:
# plot the number of cells over time
plt.figure(figsize=(8, 2))
plt.plot(df_2_total['time'], df_2_total['cells'], label='Cells', color='gray')


# estimate a linear fit
x = df_2_total['time']
y = df_2_total['cells']

A = np.vstack([x, np.ones(len(x))]).T
m, c = np.linalg.lstsq(A, y, rcond=None)[0]
print(f"m={m}, c={c}")

# dashed
plt.plot(x, m*x + c, 'r', label='Fitted line', linestyle='dashed')

# add text with the equation
plt.text(0, 5*10e13, f'N = {m:.4g}t + {c:.2g}', fontsize=12)

plt.xlabel('Time')
plt.ylabel('Number of Cells')
plt.savefig('../../report/graphics/task2-2-total-linear.pdf', bbox_inches='tight')

m=8305977.950969492, c=0.16495888081170768


In [12]:
# Load all .csv file
data = {}
for file in os.listdir("../output/bulk-t"):
    if file.endswith(".csv"):
        name = file.split(".")[0]
        record = {}
        record["df"] = pd.read_csv(f"../output/bulk-t/{file}")
        record["t"] = int(name[8:])
        print(record["t"])
        data[name] = record

# scale time
for name, record in data.items():
    dt = 0.001
    record['df']['time'] = record['df']['time'] * dt

# sort by t (in reverse order)
data = dict(sorted(data.items(), key=lambda x: x[1]['t'], reverse=True))

# get the final number of cells for each t
final_cells = []
for name, record in data.items():
    final_cells.append(record['df']['cells'].iloc[-1])

# calculate the percentage full using 10^13 as the full capacity
full_capacity = 10 ** 13
percentage_full = [x / full_capacity for x in final_cells]

# print in table format
print("T & Cells & \% Full \\\\")
for i, (name, record) in enumerate(data.items()):
    print(f"{record['t']} & {final_cells[i]:.2e} & {percentage_full[i]:.4f}\% \\\\")
print()


2000
1800
1200
1000
1400
1600
T & Cells & \% Full \\
2000 & 1.00e+13 & 0.9999\% \\
1800 & 1.00e+13 & 0.9998\% \\
1600 & 9.99e+12 & 0.9994\% \\
1400 & 9.98e+12 & 0.9979\% \\
1200 & 9.93e+12 & 0.9931\% \\
1000 & 9.77e+12 & 0.9774\% \\



In [13]:
plt.figure(figsize=(8, 3))
for name, record in data.items():
    plt.plot(record['df']['time'], record['df']['cells'], label=record["t"])
plt.xlabel('Time')
plt.ylabel('Number of Cells')
plt.yscale('log')
plt.savefig('../../report/graphics/task2-1-t.pdf', bbox_inches='tight')

In [14]:
# plot percentage full over time for smallest t
plt.figure(figsize=(8, 2))

plt.plot(data['task2-1-2000']['df']['time'], data['task2-1-2000']['df']['cells'] /2)
plt.ylabel('Number of Cells')
plt.xlabel('Time')
plt.twinx()

plt.plot(data['task2-1-2000']['df']['time'], data['task2-1-2000']['df']['cells'] / full_capacity, label='T=1', linestyle='dashed', color='red')
plt.ylabel('Percentage Full')
plt.legend(title='T')
plt.savefig('../../report/graphics/task2-1-t-percentage.pdf', bbox_inches='tight')

In [15]:
x = np.linspace(0, 2000, 2000)
y = 10 ** (13 - (4 * (np.e ** (-0.006 * x))))
analytical = pd.DataFrame({'time': x, 'cells': y})

# calculate dt/DN for each t
dtdN = 0.006 * y * np.log(10 ** 13 / y)
dt2dN = 0.006 * (np.log(10 ** 13 / y) - 1)

# print rates at t=1200
print(f"t=1200: dN/dt={dtdN[1200]:.4g}, d^2N/dt^2={dt2dN[1200]:.4g}")

# plot dt/DN for each t
plt.figure(figsize=(8,  2))
plt.plot(analytical['time'], dtdN, color='red')
plt.ylabel('$dN/dt$')
plt.xlabel('Time')
plt.twinx()
plt.plot(analytical['time'], dt2dN, color='blue')
plt.ylabel('$d^2N/dt^2$')

# draw line at t=1200
plt.axvline(x=1200, color='black', linestyle='dashed')
plt.xlabel('Time')

plt.savefig('../../report/graphics/task2-1-t-dt.pdf', bbox_inches='tight')

t=1200: dN/dt=4.083e+08, d^2N/dt^2=-0.005959


In [16]:
# plot both on the same graph
plt.figure(figsize=(8, 2))
plt.plot(data['task2-1-2000']['df']['time'], data['task2-1-2000']['df']['cells'] / full_capacity, label='T=1', color='blue')
plt.xlabel('Time')
plt.ylabel('Percentage Full')

# different y axis
plt.twinx()
plt.plot(analytical['time'], dtdN, color='red')

plt.ylabel('$dN/dt$')

plt.legend(handles=[plt.Line2D([0], [0], color='blue', label='Percentage Full'), plt.Line2D([0], [0], color='red', label='dt/dN')], loc='center right')
plt.axvline(x=1200, color='black', linestyle='dashed')
plt.savefig('../../report/graphics/task2-1-t-combined.pdf', bbox_inches='tight')